In [1]:

#%%
import pandas as pd
import os
import datetime
from collections import Counter
#%%

In [2]:

from driver import Driver


In [3]:
#%%
TICKET_FOLDER = "/".join([os.getcwd(), "tickets/"])
INSTANCE_FOLDER = "/".join([os.getcwd(), "instances/"])
#%%

""" Load dataframes """

In [4]:
df_tickets = pd.read_excel(f"{os.getcwd()}/DTICKETHISTAB_BETON.xlsx",engine='openpyxl')

In [5]:
#%%
df_orders = pd.read_excel(f"{os.getcwd()}/DORDERSTAB_BETON.xlsx",sheet_name='data',engine='openpyxl')

In [6]:
df_employee = pd.read_excel(f"{os.getcwd()}/EMPLOYETAB.xlsx",engine='openpyxl')
df_employee.drop(['COMP_NBR', 'emp_sch_specificite'], axis=1, inplace=True)
df_employee.set_index('EMPL_NBR',inplace=True)
#%%

In [7]:
df_depots = pd.read_excel(
    f"{os.getcwd()}/adresses_complete.xlsx",engine='openpyxl')
df_depots = df_depots[df_depots['Location_type'] != 'customer']
df_depots.dropna(axis=1,  inplace=True)
df_depots['NBR'] = df_depots['JOB_DESC'].apply(
    lambda x: x if len(x.split(' ')) == 1 else int(x.split(' ')[1]))
df_depots.set_index('NBR',inplace=True)
#%%

In [8]:
# Depot location
df_depots_loc = pd.read_excel( f"{os.getcwd()}/LOCATIONTAB.xlsx",engine='openpyxl')
df_depots_loc['LOC_NBR'] = df_depots_loc['LOC_NBR'].apply(lambda x: int(x))
df_depots_loc.set_index('LOC_NBR',inplace=True)
#%%

In [9]:

# Les tickets contiennent les séquences des opérations de distribution
# Un ticket est la livraison de béton sur un chantier par un livreur

# Je dois déterminer la séquence de livraison par jour.
df_tickets['DATE'] = df_tickets['TICKET_DATE'].apply(lambda x:datetime.date(x.year,x.month,x.day))
df_orders['DATE'] = df_orders['SHIPDATE'].apply(lambda x:datetime.date(x.year,x.month,x.day))
#%% Clean data
df_tickets.dropna(axis=0,inplace=True)
# # Enlever TRUCK_NBR = ACHATEXT
df_tickets = df_tickets[df_tickets['TRUCK_NBR'] != 'ACHATEXT']
# # Enlever livraison dont cap > 12
df_tickets = df_tickets[df_tickets['QUANTITY']<=12]
# # SHIP_LOC = 099
df_tickets = df_tickets[df_tickets['SHIP_LOC']!=99]

df_tickets = df_tickets[df_tickets['DRIVER_NBR']!=1]
df_tickets = df_tickets[df_tickets['DRIVER_NBR']!=999]



 Dans mes fichiers d'instance je dois mettre les chauffeurs réellement disponibles dans la journée.
   Mais dans le fichier ticket les numeros des chauffeurs ne sont pas tous présents.
   Mais si j'arrive à trouver la capacité de chaque véhicule utilisé ça aidera 

   Il y a des véhicules de capacité > 8 dans les données!!! 

   Il y a des véhicules qui n'existent pas dans le fichier des véhicules

   Il y a des employés qui n'existent pas dans le fichier des employés

In [10]:
# Liste des livreurs disponibles dans le fichier Ticket.
df_driver = df_tickets.loc[:,("DTICKETHIS_TICKET_ID","SHIP_LOC","ORDER_ID","DRIVER_NBR","TRUCK_NBR","SCHED_LOC","LOAD","LOADNR","QUANTITY")]
# Enlever les ordres où le numero du chauffeur/camion est manquant
df_driver.dropna(axis=0,inplace=True)
 # Enlever TRUCK_NBR = ACHATEXT
df_driver = df_driver[df_driver['TRUCK_NBR'] != 'ACHATEXT']
# Enlever livraison dont cap > 12
df_driver = df_driver[df_driver['QUANTITY']<=12]
# print(len(df_driver))
list_driver_nbr = list(dict.fromkeys(df_driver['DRIVER_NBR']))
#%%
employee_nbr=df_employee.index
dummy_driver=[]
for nbr in list_driver_nbr:
    if nbr not in employee_nbr:
       dummy_driver.append(int(nbr))

 Recuperer capacité des livreurs disponibles.


In [11]:
list_driver = []
dict_driver = {}
list_cap = []
for nbr in list_driver_nbr:
    d = Driver() 
    d.nbr = int(nbr)
    _df =  df_driver[df_driver['DRIVER_NBR']==nbr]
    _cap = max(_df['QUANTITY']) 
    if _cap <= 8:
        d.capacity = 8
    else:
        d.capacity = 12 
    d.factory_nbr =  Counter(_df['SHIP_LOC']).most_common(1)[0][0]
    list_cap.append(d.capacity)
    list_driver.append(d)
    dict_driver[d.nbr] = d

#%%

In [12]:
"""
Matcher les livreurs de list_driver avec les employés de CQ
"""
for i,emp_nbr in enumerate(df_employee.index):
    emp = df_employee.loc[emp_nbr]
    if emp_nbr in dict_driver:
        d = dict_driver[emp_nbr]
        d.id = i
        d.rank = emp['emp_rang']
        d.description = emp['sch_description']
        d.depot_nbr = emp['emp_usi_numero']
    else:
        d = Driver()
        d.id = i
        d.nbr = emp_nbr
        d.rank = emp['emp_rang']
        d.description = emp['sch_description']
        d.capacity = 8
        if 'semi' in d.description:
            d.capacity = 12   
        d.factory_nbr = d.depot_nbr = emp['emp_usi_numero']
        dict_driver[d.nbr] = d
        list_driver.append(d)

#%%

In [13]:
# Find dummy driver (occasional drivers)
#%%
print(len(dummy_driver))
#%%

15


In [25]:
jour = datetime.date(2020,11,28)
day_op = df_tickets[df_tickets['DATE']==jour]
day_op = day_op[["DTICKETHIS_TICKET_ID","SHIP_LOC","ORDER_ID","DRIVER_NBR","TRUCK_NBR","SCHED_LOC","LOAD","LOADNR","QUANTITY","BEGIN_LOAD","FINISH_LOAD","ARRIVE_PLANT"]]
orders_of_day = list(dict.fromkeys(day_op['ORDER_ID']))
all_orders_of_day = list(dict.fromkeys(df_orders['ORDER_ID'][df_orders['DATE'] == jour ]))
# print(f"Nombre ordres dans Ticket table: {len(orders_of_day)} ")
# print(f"Nombre ordres dans Order table: {len(all_orders_of_day)} ")
real_orders_of_day = list(set(orders_of_day) & set(all_orders_of_day))
print(f"{len(real_orders_of_day)} ordres servis")
for i,ord in enumerate(real_orders_of_day):
            # print(f"Operations for order {ord}")
            ord_seq = day_op[day_op['ORDER_ID']==ord]
            ordres = df_orders[df_orders['ORDER_ID']==ord]
            if len(ordres) == 0: 
                continue     
                
            ordre = ordres.iloc[0]
            string = f"""{i} {ord} {ordre['CUST_NBR']} {ordre['LOCATION ID']} {ordre['SCHED_LOC']} {ordre['SHIPTIME (min)']:.2f} {ordres['QUANTITY'].sum():.2f} \n"""
            print(string)
           
#           contents.append(string)       

5 ordres servis
0 925444 104686 68 54 420.00 12.00 

1 928300 103762 850 54 660.00 13.00 

2 927569 003483 146 54 445.00 221.00 

3 927089 104686 617 54 600.00 3.00 

4 928061 104686 807 54 540.00 18.00 



In [14]:


ticket_date = list(dict.fromkeys(df_tickets['DATE']))
#%%
## Dans le fichier des tickets, récuperer la liste des chauffeurs qui ont effectué les livraisons
# creer un fichier où enregistrer les dépôts, ordres et chauffeurs.
instance_size = dict()
ticket_date = [datetime.date(2020,11,7)]
for jour in ticket_date:
    day_op = df_tickets[df_tickets['DATE']==jour]
    day_op = day_op[["DTICKETHIS_TICKET_ID","SHIP_LOC","ORDER_ID","DRIVER_NBR","TRUCK_NBR","SCHED_LOC","LOAD","LOADNR","QUANTITY","BEGIN_LOAD","FINISH_LOAD","ARRIVE_PLANT"]]
    day = jour.day
    if day < 10:
        day = f"0{day}"
    with open(f"{INSTANCE_FOLDER}/{jour.year}{jour.month}{day}.txt", 'w') as file:
        contents = []
        order_count = 0
        contents.append(f"{len(df_depots)}\n")
        orders_of_day = list(dict.fromkeys(day_op['ORDER_ID']))
        all_orders_of_day = list(dict.fromkeys(df_orders['ORDER_ID'][df_orders['DATE'] == jour ]))
        # print(f"Nombre ordres dans Ticket table: {len(orders_of_day)} ")
        # print(f"Nombre ordres dans Order table: {len(all_orders_of_day)} ")
        real_orders_of_day = list(set(orders_of_day) & set(all_orders_of_day))
        print(f"{len(real_orders_of_day)} ordres servis")
        
        # 2 Nombre de clients
        contents.append(f"{len(real_orders_of_day)}\n")

        list_driver = list(dict.fromkeys(day_op['DRIVER_NBR']))
        # print(f"Nombre livreurs utilisés: {len(list_driver)}")
        list_trucks = list(dict.fromkeys(day_op['TRUCK_NBR']))
        # print(f"Nombre trucks utilisés: {len(list_trucks)}")
        # 3 Nombre de chauffeurs
        # file.write(f"{len(list_driver)}\n")
#         check=any(item in list_driver for item in dummy_driver)
#         if  not check:
#            continue

        print(jour)

        contents.append(f"{len(list_driver)}\n")

        # 4 Liste des depots
        # for i,d in enumerate(list_depot):
        #     contents.append(f"{i} {df_depots.loc[d]['Location ID']} {d} {df_depots_loc.loc[d]['CAPACITY']}\n")  
        for i, dep in enumerate(df_depots.index):
            contents.append(
                f"{i} {df_depots.loc[dep]['Location ID']} {dep} {df_depots_loc.loc[dep]['CAPACITY']} \n")
        
        # 5 Liste des clients de la journée  
        for i,ord in enumerate(real_orders_of_day):
            # print(f"Operations for order {ord}")
            ord_seq = day_op[day_op['ORDER_ID']==ord]
            ordres = df_orders[df_orders['ORDER_ID']==ord]
            if len(ordres) == 0: 
                continue     
                
            ordre = ordres.iloc[0]
            string = f"""{i} {ord} {ordre['CUST_NBR']} {ordre['LOCATION ID']} {ordre['SCHED_LOC']} {ordre['SHIPTIME (min)']:.2f} {ordres['QUANTITY'].sum():.2f} \n"""
#             print(string)
            contents.append(string)
            
        # 6 Liste des ordres de la journée  
        for i,ord in enumerate(real_orders_of_day):
            # print(f"Operations for order {ord}")
            ord_seq = day_op[day_op['ORDER_ID']==ord]
            ordres = df_orders[df_orders['ORDER_ID']==ord]
            if len(ordres) == 0: 
                continue          
            # Pouring minutes
            ord_seq_ticket_id = list(dict.fromkeys(ord_seq['DTICKETHIS_TICKET_ID'] ))
            # print(ord_seq_ticket_id)
            # filter le ticket selon ord
            # Get ticket for ord_seq
            pouring_duration = []
            pouring_minutes = 3
            for id in ord_seq_ticket_id:
                ticket_schedule =  ord_seq[ord_seq['DTICKETHIS_TICKET_ID'] == id]
                temp = ticket_schedule["FINISH_LOAD"]-ticket_schedule["BEGIN_LOAD"]
                _max = max(temp)
                if _max.days >= 0:
                    pouring_duration.append(_max)
            if len(pouring_duration) > 0:
                pouring_minutes = round(max(pouring_duration).seconds/60,2)
            if pouring_minutes < 0.5:
                pouring_minutes = 3
            for j in range(len(ordres)):
                ordre = ordres.iloc[j]
                # print(f"{i} {ordre['CUST_NBR']}  {ordre['SCHED_LOC']} {ordre['SHIPTIME (min)']:.2f} {ordre['QUANTITY']:.2f} {ordre['TRAVEL_MINUTES']} {ordre['UNLOAD_MINUTES']} {pouring_minutes} \n") 
                # file.write(f"{order_count} {ord} {ordre['CUST_NBR']} {ordre['SCHED_LOC']} {ordre['SHIPTIME (min)']:.2f} {ordre['QUANTITY']:.2f} {ordre['TRAVEL_MINUTES']} {ordre['UNLOAD_MINUTES']} {pouring_minutes} \n")
                contents.append(f"{order_count} {i} {ordre['QUANTITY']:.2f} {ordre['TRAVEL_MINUTES']} {ordre['UNLOAD_MINUTES']} {pouring_minutes} \n")

                order_count = order_count + 1
            
        # 7 Ajouter les chauffeurs
        # Lines of drivers: driver id, driver number, driver rank, depot assigned driver max capacity
        for i,id in enumerate(list_driver):
            dr = dict_driver[id]
            df_drv =day_op[day_op['DRIVER_NBR']==dr.nbr] 
            _data = df_drv[["BEGIN_LOAD","ARRIVE_PLANT"]]
            _data.sort_values(["BEGIN_LOAD"],inplace=True)
            # print(dr)
            # _data = test[]
            begin, end = _data.iloc[0,0],_data.iloc[-1,1]
            factory_nbr = Counter(df_drv['SHIP_LOC']).most_common(1)[0][0]
            contents.append(f"{i} {dr.nbr} {dr.rank} {dr.depot_nbr} {factory_nbr} {dr.capacity:.0f} {begin.hour*60 + begin.minute} {end.hour*60 + end.minute} \n")
            # print(end)
            # break

        # print(order_count)
        contents.insert(2,str(order_count)+'\n')
        contents[-1] = contents[-1].rstrip()
        contents = "".join(contents)
        file.write(contents)
        instance_size[f"{jour.year}{jour.month}{day}"] = [len(df_depots), len(real_orders_of_day), order_count, len(list_driver)]
        
pd.DataFrame.from_dict(instance_size, orient='index').to_csv(f'instances/instance_size.csv')


6 ordres servis
2020-11-07


/var/folders/ym/09bxjqq91yn1yjvfy33yk4nw0000gn/T/ipykernel_16093/1500297298.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _data.sort_values(["BEGIN_LOAD"],inplace=True)
/var/folders/ym/09bxjqq91yn1yjvfy33yk4nw0000gn/T/ipykernel_16093/1500297298.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _data.sort_values(["BEGIN_LOAD"],inplace=True)
/var/folders/ym/09bxjqq91yn1yjvfy33yk4nw0000gn/T/ipykernel_16093/1500297298.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/index

In [34]:
instance_size

{'20201001': [8, 98, 113, 116],
 '20201002': [8, 77, 85, 119],
 '20201003': [8, 10, 14, 31],
 '20201005': [8, 82, 88, 107],
 '20201006': [8, 113, 135, 123],
 '20201007': [8, 64, 78, 123],
 '20201008': [8, 99, 118, 122],
 '20201009': [8, 96, 114, 112],
 '20201010': [8, 5, 8, 13],
 '20201013': [8, 42, 47, 98],
 '20201014': [8, 109, 126, 122],
 '20201015': [8, 88, 109, 122],
 '20201016': [8, 71, 82, 111],
 '20201017': [8, 6, 6, 28],
 '20201019': [8, 92, 105, 120],
 '20201020': [8, 85, 97, 132],
 '20201021': [8, 40, 43, 78],
 '20201022': [8, 107, 123, 135],
 '20201023': [8, 112, 126, 130],
 '20201024': [8, 9, 10, 36],
 '20201026': [8, 67, 78, 116],
 '20201027': [8, 99, 115, 131],
 '20201028': [8, 114, 127, 134],
 '20201029': [8, 121, 130, 130],
 '20201030': [8, 93, 116, 132],
 '20201031': [8, 4, 7, 15],
 '20201102': [8, 95, 106, 124],
 '20201103': [8, 89, 108, 123],
 '20201104': [8, 110, 129, 127],
 '20201105': [8, 107, 122, 127],
 '20201106': [8, 111, 133, 123],
 '20201109': [8, 63, 70, 9